In [67]:
## Here we will write out the matrix elements for our non-eq setup

from qutip import *
import numpy as np
from scipy import integrate
from helper_code_qutip import * 
import scipy.io

In [68]:
## Firstly, we have to define the integral function C and D

def integral1(i,k,tb,beta,mu,gamma,eigenergies,limit_value = 700,b=50):
    freq=eigenergies[k]-eigenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta,mu,gamma),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta,mu,gamma),limit=limit_value)[0]
    return integral

def integral2(i,k,tb,gamma,eigenergies,limit_value = 700,b=50):
    freq=eigenergies[k]-eigenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma),limit=limit_value)[0]
    return integral

def C(i,k,tb,beta,mu,gamma,eigenergies):
    val = integral1(i,k,tb,beta,mu,gamma,eigenergies) + 0.5*(func1(eigenergies[k]-eigenergies[i],tb,beta,mu,gamma))

    return val

def D(i,k,tb,beta,mu,gamma,eigenergies):
    val = integral1(i,k,tb,beta,mu,gamma,eigenergies) + integral2(i,k,tb,gamma,eigenergies) + 0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma)+func1(eigenergies[k]-eigenergies[i],tb,beta,mu,gamma))
    return val

In [69]:
NL = 3
NM = 2

N = NL + NM
dL = 2**NL
dM = 2**NM
d = 2**N
dims = [2]*N

In [70]:
## Define the Hamiltonian 

w0list = np.linspace(1,1,N)
glist = np.linspace(0.0016,0.0016,N-1)

delta = 1

H_S = create_hamiltonian(w0list,glist,delta,N)

create_sm_list = [create_sm(N,i + 1) for i in range(NL)]  # Create list of sigma_- operators

eigenergies,eigstates=H_S.eigenstates()

In [71]:
## Let us compute the thermal density matrix
beta = 1

rho_th = (-beta*H_S).expm()/((-beta*H_S).expm()).tr() 
print(rho_th)

Quantum object: dims=[[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape=(32, 32), type='oper', dtype=Dense, isherm=True
Qobj data =
[[1.41406127e-03 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 3.83155610e-03 1.22413620e-05 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.22413620e-05 3.81933431e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 7.67133803e-02
  2.45874329e-04 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.45874329e-04
  7.69588615e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 2.09865300e-01]]


In [72]:
## Now we set parameters for the  non-eq setup

beta_list = np.linspace(1,1,NL)
mu_list = np.linspace(-0.5,-0.5,NL)

gamma_list = np.linspace(1,1,NL)

tb = 0.01

In [73]:
number = len(eigenergies)

## Now we will write out the matrix elements

A = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        sum = 0
        vi = eigstates[i]
        vk = eigstates[k]
        proj_i = vi*vi.dag()
        proj_k = vk*vk.dag()
        for y in range(number):
            for l in range(NL):
                proj_y = eigstates[y]*eigstates[y].dag()
                op1 = commutator(proj_k*create_sm_list[l]*proj_y,create_sm_list[l].dag())*C(k,y,tb,beta_list[l],mu_list[l],gamma_list[l],eigenergies)
                sum += vi.dag()*(op1 + op1.dag())*vi

                op2 = commutator(create_sm_list[l].dag(),proj_y*create_sm_list[l]*proj_k)*D(y,k,tb,beta_list[l],mu_list[l],gamma_list[l],eigenergies)
                sum += vi.dag()*(op2 + op2.dag())*vi

        A[i,k] = sum


In [74]:
print(number)  #should be 2^N

32


In [75]:
rho_th_new = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        vi = eigstates[i]
        vk = eigstates[k]

        rho_th_new[i,k] = vi.dag()*rho_th*vk

#print(rho_th_new)

rho_th_diag = [rho_th_new[i,i] for i in range(number)]
print(rho_th_diag)


[(0.2098653003484718+0j), (0.07720512941347273+0j), (0.07711081990899829+0j), (0.0768644596718962+0j), (0.07656102913367026+0j), (0.07631642541512064+0j), (0.028402179864197227+0j), (0.028367485336394015+0j), (0.028327515430261768+0j), (0.028276854470042037+0j), (0.02825841760148131+0j), (0.028165228613205123+0j), (0.028101885087261656+0j), (0.028075243933916638+0j), (0.02800380558258656+0j), (0.027874921844230863+0j), (0.010448578056491667+0j), (0.010435814652991707+0j), (0.010421110546260104+0j), (0.010402473420525266+0j), (0.010395690875622194+0j), (0.010361408562691818+0j), (0.010338105781765906+0j), (0.010328305049161178+0j), (0.010302024348395661+0j), (0.010254610670753275+0j), (0.00384381705645835+0j), (0.0038391216627113407+0j), (0.003826856108743618+0j), (0.0038117491917920637+0j), (0.0037995710897336036+0j), (0.0014140612706951565+0j)]


In [76]:
## Now we use the lin algebra solver to solve the equation Ax = b, where b is all zeroes

b = np.zeros((number),dtype=complex)

x = np.linalg.solve(A,b)

print(x)

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j]


In [77]:
print(A)

[[ 8.51668227e-001+0.00000000e+000j -7.64645337e-001+0.00000000e+000j
  -6.37743940e-001+0.00000000e+000j ...  0.00000000e+000+0.00000000e+000j
  -2.54916080e-102+0.00000000e+000j  0.00000000e+000+0.00000000e+000j]
 [-1.70615436e-001+1.28749008e-019j  1.44626171e+000-5.57347679e-017j
   9.74784573e-017-1.38026188e-017j ... -4.92662509e-033-3.87287780e-049j
  -2.67101774e-032+2.57256864e-050j -8.71454513e-078+1.60113493e-093j]
 [-1.42126082e-001+1.49077799e-019j  9.83892022e-017+2.79407866e-017j
   1.34772409e+000+2.82299141e-017j ... -1.90288977e-032-9.26995513e-049j
  -5.05199383e-032-1.54374853e-048j -1.42379308e-076+3.37728704e-091j]
 ...
 [ 0.00000000e+000+0.00000000e+000j -1.15752302e-034+1.73201663e-050j
  -4.51151517e-034-4.13805703e-051j ...  3.32504262e+000-1.91903785e-017j
   1.24280275e-016+3.74794130e-017j -6.33502320e-001+8.32667268e-017j]
 [ 3.87676282e-089+8.81973918e-104j -5.38832940e-034-2.36785149e-050j
  -1.02726686e-033-5.54589351e-050j ...  5.43984439e-018+5.020008

In [78]:
## Now we can add one more constraint, that the sum of all the elements of x should be 1, which is basically another row in the matrix A consisting of all 1

#A = np.vstack([A,np.ones((1,number))])

#print(A)

print(np.linalg.matrix_rank(A))

q,r = np.linalg.qr(A.T)

print(r)
diag_r = [r[i][i] for i in range(number)]

print(np.linalg.matrix_rank(A))





31
[[-1.93308203e+00+0.00000000e+00j  6.47248797e-01-3.60036069e-17j
   5.07245489e-01+2.69789966e-17j ... -1.16339352e-33+1.14733441e-50j
  -1.49444846e-33-2.72903061e-50j -2.32342253e-66+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j -2.01798942e+00+0.00000000e+00j
  -1.17527708e-01-1.66129574e-17j ... -4.29571555e-31+2.26112848e-47j
  -2.89061570e-31+6.16995632e-49j -8.44494456e-33-2.11423086e-49j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  -1.82567814e+00+0.00000000e+00j ... -5.88821619e-31+1.29137639e-47j
  -3.58560029e-31-1.33752811e-47j -1.27082711e-32+3.50152517e-49j]
 ...
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j ... -2.63062169e+00+0.00000000e+00j
   9.49564646e-01+1.25089221e-16j  1.68105705e+00-6.35959191e-17j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j ...  0.00000000e+00+0.00000000e+00j
  -2.52345551e+00+0.00000000e+00j  2.5234555

In [79]:
print(diag_r)

[(-1.9330820293652171+0j), (-2.0179894164512655+0j), (-1.8256781439074603+0j), (-1.714327001980009+0j), (-1.6806389375843538+0j), (-1.7181724580180595+0j), (-2.0969337794766774+0j), (-2.0863080447354188+0j), (-2.0847889751645132+0j), (-2.0799407800610688+0j), (-2.1170090872056426+0j), (-2.05239087001904+0j), (-1.9899026996652756+0j), (-2.089012844806579+0j), (-2.088226829374746+0j), (-2.0175527816393477+0j), (-2.1502702807834795+0j), (-2.4096268441487316+0j), (-2.295695676638247+0j), (-2.4589337137231237+0j), (-2.388456575435351+0j), (-2.4142452966721777+0j), (-2.5283374448671183+0j), (-2.452412753231627+0j), (-2.4558237795728006+0j), (-2.3447715663885322+0j), (-2.180832665476792+0j), (-2.6159239786963076+0j), (-2.7449544795952505+0j), (-2.6306216927662724+0j), (-2.5234555103276084+0j), (1.2753340199279043e-15+0j)]


In [80]:
## Since last element of r is 0, we can remove the last row of  A and add a row of 1's

A_new = A[:-1]
A_new = np.vstack([A_new,np.ones((1,number))])

print(np.linalg.matrix_rank(A_new))
print(A_new.shape)
print(A_new)

32
(32, 32)
[[ 8.51668227e-001+0.00000000e+000j -7.64645337e-001+0.00000000e+000j
  -6.37743940e-001+0.00000000e+000j ...  0.00000000e+000+0.00000000e+000j
  -2.54916080e-102+0.00000000e+000j  0.00000000e+000+0.00000000e+000j]
 [-1.70615436e-001+1.28749008e-019j  1.44626171e+000-5.57347679e-017j
   9.74784573e-017-1.38026188e-017j ... -4.92662509e-033-3.87287780e-049j
  -2.67101774e-032+2.57256864e-050j -8.71454513e-078+1.60113493e-093j]
 [-1.42126082e-001+1.49077799e-019j  9.83892022e-017+2.79407866e-017j
   1.34772409e+000+2.82299141e-017j ... -1.90288977e-032-9.26995513e-049j
  -5.05199383e-032-1.54374853e-048j -1.42379308e-076+3.37728704e-091j]
 ...
 [ 0.00000000e+000+0.00000000e+000j -1.15752302e-034+1.73201663e-050j
  -4.51151517e-034-4.13805703e-051j ...  3.32504262e+000-1.91903785e-017j
   1.24280275e-016+3.74794130e-017j -6.33502320e-001+8.32667268e-017j]
 [ 3.87676282e-089+8.81973918e-104j -5.38832940e-034-2.36785149e-050j
  -1.02726686e-033-5.54589351e-050j ...  5.43984439e-

In [81]:
print(A.shape)

(32, 32)


In [82]:
print(np.linalg.matrix_rank(A))
print(A.shape)

31
(32, 32)


In [83]:
b[-1] = 1  ## Last element of b is 1

x = np.linalg.solve(A_new,b)
print(x)

[3.66681937e-01-2.10053432e-18j 8.18177994e-02+7.52443898e-18j
 8.17178553e-02-3.83190620e-18j 8.14567761e-02-2.11972370e-18j
 8.11352170e-02+1.93354517e-18j 8.08759992e-02-4.93883002e-18j
 1.82560187e-02+2.39971276e-18j 1.82337181e-02+4.75233936e-19j
 1.82080268e-02+3.11970985e-19j 1.81754635e-02+3.27010316e-19j
 1.81636129e-02+3.10529134e-20j 1.81037140e-02+4.43070974e-20j
 1.80629987e-02-1.79698895e-19j 1.80458747e-02+2.37600008e-19j
 1.79999564e-02-1.97982230e-18j 1.79171140e-02+2.47085527e-19j
 4.07346837e-03+5.27310842e-19j 4.06849245e-03+4.66741185e-20j
 4.06275993e-03-6.10212758e-20j 4.05549408e-03+2.36081366e-19j
 4.05284985e-03-1.22938888e-19j 4.03948459e-03-6.88979719e-21j
 4.03039980e-03-5.06205112e-20j 4.02657890e-03+2.11920912e-19j
 4.01633314e-03-1.16494636e-19j 3.99784851e-03+3.17277909e-19j
 9.08913651e-04+1.75911737e-19j 9.07803372e-04+7.29424730e-20j
 9.04903044e-04+5.52700488e-20j 9.01330844e-04+4.22022491e-20j
 8.98451195e-04+6.25715198e-20j 2.02806048e-04+2.283596

In [84]:
##Let us check if this is correct

print(np.dot(A_new,x))
print(np.dot(A[-1],x))

[-2.77555756e-16-4.81482486e-33j  4.16333634e-17+3.85185989e-33j
  6.24500451e-17-7.70371978e-34j  6.93889390e-17-3.85185989e-34j
 -6.93889390e-18-1.15555797e-33j  6.93889390e-17+7.70371978e-34j
  3.46944695e-18-3.85185989e-34j  0.00000000e+00+2.40741243e-34j
  5.20417043e-18+6.74075481e-34j  3.46944695e-18-3.85185989e-34j
 -1.90819582e-17-9.62964972e-34j -6.93889390e-18-2.40741243e-34j
 -1.38777878e-17-4.81482486e-35j -7.80625564e-18+6.50001356e-34j
  5.20417043e-18+7.70371978e-34j -1.73472348e-18+1.92592994e-34j
  1.73472348e-18-1.92592994e-34j -2.60208521e-18+0.00000000e+00j
  7.37257477e-18-4.81482486e-35j  1.73472348e-18-9.62964972e-35j
  8.67361738e-19-3.61111865e-35j -8.67361738e-19+2.40741243e-35j
  0.00000000e+00-9.62964972e-35j  4.33680869e-19+7.22223729e-35j
 -4.33680869e-18+0.00000000e+00j  0.00000000e+00+2.88889492e-34j
  1.73472348e-18+9.62964972e-35j  6.50521303e-19-1.20370622e-35j
 -2.16840434e-19+0.00000000e+00j  0.00000000e+00-1.80555932e-35j
 -1.08420217e-18-9.629649

In [85]:
#We have to tranfer the basis of the solution rho matrix to computational basis from the eigenbasis

x_real = [np.real(x[i]) for i in range(number)]

rho = np.diag(x_real)

#set U matrix whose columns are the eigenvectors of the Hamiltonian

U = np.zeros((number,number),dtype=complex)
for i in range(number):
    U[:,i] = eigstates[i].full().flatten()

check = 1

print(U[:,check])
print(eigstates[check])

[ 0.00000000e+00+0.j  9.87319092e-63+0.j  9.26598968e-60+0.j
 -3.36732947e-33+0.j  0.00000000e+00+0.j  1.30637770e-30+0.j
  2.43826455e-17+0.j -3.51896464e-18+0.j  1.17698862e-46+0.j
 -2.43826455e-17+0.j  5.61708723e-18+0.j -6.26841772e-17+0.j
  9.02735333e-17+0.j -5.20465912e-16+0.j -1.79659379e-16+0.j
 -4.47213595e-01+0.j -8.64264045e-45+0.j -3.56168200e-17+0.j
 -4.64377198e-17+0.j  7.08919476e-16+0.j  8.57477382e-18+0.j
 -1.74234308e-16+0.j  4.11320628e-16+0.j -4.47213595e-01+0.j
 -1.94417344e-17+0.j -6.54098945e-16+0.j  6.86697194e-16+0.j
 -4.47213595e-01+0.j  7.29560898e-16+0.j -4.47213596e-01+0.j
 -4.47213596e-01+0.j  0.00000000e+00+0.j]
Quantum object: dims=[[2, 2, 2, 2, 2], [1, 1, 1, 1, 1]], shape=(32, 1), type='ket', dtype=Dense
Qobj data =
[[ 0.00000000e+00]
 [ 9.87319092e-63]
 [ 9.26598968e-60]
 [-3.36732947e-33]
 [ 0.00000000e+00]
 [ 1.30637770e-30]
 [ 2.43826455e-17]
 [-3.51896464e-18]
 [ 1.17698862e-46]
 [-2.43826455e-17]
 [ 5.61708723e-18]
 [-6.26841772e-17]
 [ 9.0273533

In [86]:
rho_comp = np.dot(np.conjugate(U.T),np.dot(rho,U))

print(rho_comp)

[[ 2.02806048e-04+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  4.93025566e-03+0.j  5.09758507e-03+0.j ...
  -6.65739415e-38+0.j  3.91159839e-41+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  5.09758507e-03+0.j  7.48855581e-03+0.j ...
   4.64312380e-38+0.j  3.22661430e-40+0.j  0.00000000e+00+0.j]
 ...
 [ 0.00000000e+00+0.j -6.65739415e-38+0.j  4.64312380e-38+0.j ...
   4.80234051e-02+0.j -2.51947717e-02+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  3.91159839e-41+0.j  3.22661430e-40+0.j ...
  -2.51947717e-02+0.j  6.18957768e-02+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  3.66681937e-01+0.j]]


In [87]:
def L2_red(rho,eigenergies,eigstates,beta_list,mu_list,gamma_list,tb):
    sum = 0
    rho = Qobj(rho)
    rho.dims = [dims,dims]
    for i in range(number):
        for k in range(number):
            vi = eigstates[i]
            vk = eigstates[k]

            proj_i = vi*vi.dag()
            proj_k = vk*vk.dag()

            for l in range(NL):
                op = commutator(rho*proj_i*create_sm_list[l]*proj_k,create_sm_list[l].dag())*C(i,k,tb,beta_list[l],mu_list[l],gamma_list[l],eigenergies) + commutator(create_sm_list[l].dag(),proj_i*create_sm_list[l]*proj_k*rho)*D(i,k,tb,beta_list[l],mu_list[l],gamma_list[l],eigenergies)
                sum += op
                sum += op.dag()
    data = sum.full()
    sum = np.array(data,dtype=complex)
    return sum

In [88]:
L2_redfield = L2_red(rho_comp,eigenergies,eigstates,beta_list,mu_list,gamma_list,tb)

In [89]:
print(L2_redfield)

[[-1.07100525e-02+0.00000000e+00j -6.71408572e-03-7.94442534e-07j
  -4.82440379e-03-7.21915151e-09j ...  2.49336861e-22-5.28534400e-21j
   5.27197666e-22-2.58349980e-21j  0.00000000e+00+0.00000000e+00j]
 [-6.71408572e-03+7.94442534e-07j  4.12808199e-03+0.00000000e+00j
   1.22013066e-02-6.33378365e-07j ...  3.90921876e-22-4.45583209e-21j
   4.72842011e-22-2.52064726e-21j  1.90305309e-24-2.65861851e-24j]
 [-4.82440379e-03+7.21915151e-09j  1.22013066e-02+6.33378365e-07j
   1.54160639e-02+0.00000000e+00j ...  4.20436294e-21-8.55956260e-21j
   7.76548485e-21-2.03292417e-20j -1.38733894e-22+3.69473119e-22j]
 ...
 [ 2.49336861e-22+5.28534400e-21j  3.90921876e-22+4.45583209e-21j
   4.20436294e-21+8.55956260e-21j ... -1.42201743e-02+0.00000000e+00j
  -4.11524662e-02-1.72480935e-05j -6.90471103e-03-3.49049633e-08j]
 [ 5.27197666e-22+2.58349980e-21j  4.72842011e-22+2.52064726e-21j
   7.76548485e-21+2.03292417e-20j ... -4.11524662e-02+1.72480935e-05j
  -5.09521500e-02+0.00000000e+00j -4.92766563e-

In [90]:
## As we can see, all the checks are completely satisfied. Thus we can arrange the values of x (real values) in a diagonal density matrix and save it in a .mat file
#We will also pass in L2_redfield operator

rho_th_check = np.zeros((number,number),dtype=complex)
for i in range(number):
    for k in range(number):
        rho_th_check[i,k] = rho_th[i,k]

print(rho_th_check)

data_dict = {"dm_ness":rho_comp, "dm_th":rho_th_new,"L2_red":L2_redfield, "dm_th_check":rho_th_check}

scipy.io.savemat(F'../matlab/ness_data_NL = {NL}_NM = {NM}_1.mat',data_dict)


[[1.41406127e-03+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 3.83155610e-03+0.j 1.22413620e-05+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 1.22413620e-05+0.j 3.81933431e-03+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
 ...
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  7.67133803e-02+0.j 2.45874329e-04+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  2.45874329e-04+0.j 7.69588615e-02+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 2.09865300e-01+0.j]]
